In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from os import listdir, makedirs
from os.path import join, isfile, isdir, exists
from datetime import datetime
import pandas as pd
import numpy as np
import math
from tqdm import tqdm

In [3]:
combined_results_folder = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/montoye_thigh/combined_results_ANN_thigh_2015/'
combined_data_files =  [f for f in listdir(combined_results_folder) if isfile(join(combined_results_folder, f))]
output_folder = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/thigh_minute_data/'
ActiPAL_output_folder = output_folder
print(len(combined_data_files))


combined_ids = [f.split('_')[0] for f in listdir(combined_results_folder) if isfile(join(combined_results_folder, f))]
combined_ids = list(set(combined_ids))
print(combined_ids)

1069
['OA71', 'OA14', 'OA4', 'OA78', 'OA17', 'OA84', 'OA86', 'KOA1', 'KOA67', 'OA67', 'OA39', 'HOA39', 'KOA70', 'OA40', 'KOA38', 'KOA4', 'OA23', 'OA50', 'KOA47', 'KOA42', 'KOA69', 'KOA8', 'OA6', 'HOA29', 'HOA32', 'HOA34', 'OA1', 'HOA33', 'OA9', 'OA2', 'HOA26', 'HOA37', 'OA22', 'KOA73', 'HOA46', 'HOA41', 'KOA56', 'KOA60', 'KOA68', 'HOA44', 'OA56', 'HOA40', 'OA77', 'HOA24', 'KOA41', 'OA48', 'OA75', 'OA10', 'OA47', 'OA12', 'OA66', 'OA31', 'OA38', 'KOA3', 'HOA17', 'HOA6', 'KOA55', 'KOA25', 'OA62', 'KOA21', 'OA60', 'OA80', 'HOA35', 'OA82', 'KOA18', 'KOA61', 'OA36', 'KOA58', 'OA25', 'KOA57', 'KOA22', 'KOA50', 'HOA4', 'KOA74', 'KOA54', 'OA37', 'OA43', 'OA33', 'OA7', 'HOA9', 'KOA16', 'KOA64', 'OA74', 'OA89', 'HOA7', 'OA54', 'KOA9', 'KOA63', 'OA8', 'HOA42', 'KOA59', 'OA35', 'HOA36', 'KOA17', 'OA59', 'OA32', 'OA3', 'HOA47', 'KOA31', 'OA58', 'OA49', 'HOA14', 'OA72', 'OA61', 'HOA16', 'KOA6', 'OA63', 'KOA29', 'HOA10', 'HOA15', 'HOA2', 'OA21', 'KOA45', 'OA65', 'OA44', 'KOA27', 'KOA46', 'KOA7', 'HOA2

### Generate statistical features

In [4]:

def met_to_intensity_m1(row):
  ee = per_person_df['mont_1'][row.name]
  return 1 if ee <= 1.5 else 2 if ee < 3 else 3

def met_to_intensity_m2(row):
  ee = per_person_df['mont_2'][row.name]
  return 1 if ee <= 1.5 else 2 if ee < 3 else 3

def met_to_intensity_m3(row):
  ee = per_person_df['mont_3'][row.name]
  return 1 if ee <= 1.5 else 2 if ee < 3 else 3

def met_to_intensity_m4(row):
  ee = per_person_df['mont_4'][row.name]
  return 1 if ee <= 1.5 else 2 if ee < 3 else 3

def get_bmi_range(row):
  bmi = per_person_df['BMI'][row.name]
  if(18.5<=bmi<25):return 'normal'
  elif(25<=bmi<30):return 'overweight'
  elif(30<=bmi):return 'obese'
  else:'error'

In [ ]:
s = ['A']*10
print(s)

['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A']


In [5]:
all_df = pd.DataFrame()
for each_id in combined_ids:
    sex = ''
    bmi = ''
    complete_df = pd.DataFrame()
    # print(combined_data_files)
    for combined_file in combined_data_files:
        
        if (combined_file.split('_')[0] != each_id): continue
        print(combined_file)
        combined_data = pd.read_csv(combined_results_folder + combined_file)
        print(len(combined_data))
        sex = combined_data['sex'][0]
        if(sex!='male' and sex!='female'):
          if(each_id in combined_ids):
            print(each_id)
            sex='female'
        bmi = combined_data['BMI'][0]
        if(str(bmi)=='nan'):
          bmi = 34.18751650116381
        
        temp_df = combined_data[['predicted_ee_montoye_2015_feature_set_3',
       'predicted_ee_montoye_2015_feature_set_2',
       'predicted_ee_montoye_2015_feature_set_1',
       'predicted_ee_montoye_2015_feature_set_4']]
        
        complete_df = pd.concat([complete_df, temp_df])
        
    
    complete_df.reset_index(drop=True, inplace=True)
    complete_df_grouped = complete_df.groupby(complete_df.index // 2)
    per_person_df = pd.DataFrame()
    per_person_df['Participant'] = [each_id]*len(complete_df_grouped)
    per_person_df['sex'] = [sex]*len(complete_df_grouped)
    per_person_df['BMI'] = [bmi]*len(complete_df_grouped)
    per_person_df['mont_1'] = complete_df_grouped['predicted_ee_montoye_2015_feature_set_1'].mean()
    per_person_df['mont_2'] = complete_df_grouped['predicted_ee_montoye_2015_feature_set_2'].mean()
    per_person_df['mont_3'] = complete_df_grouped['predicted_ee_montoye_2015_feature_set_3'].mean()
    per_person_df['mont_4'] = complete_df_grouped['predicted_ee_montoye_2015_feature_set_4'].mean()
    per_person_df['mont_1_cat'] = per_person_df.apply(met_to_intensity_m1,axis=1)
    per_person_df['mont_2_cat'] = per_person_df.apply(met_to_intensity_m2,axis=1)
    per_person_df['mont_3_cat'] = per_person_df.apply(met_to_intensity_m3,axis=1)
    per_person_df['mont_4_cat'] = per_person_df.apply(met_to_intensity_m4,axis=1)
    per_person_df['BMI_range'] = per_person_df.apply(get_bmi_range,axis=1)
    all_df = pd.concat([all_df, per_person_df])
    

# print(len(all_df))
out_file = output_folder+'per_minute_EE_means_thigh_mont_15.csv'
print(out_file)
all_df.to_csv(out_file)

  


OA71_2016-08-08_15-59-48_to_2016-08-08_20-17-00_Epoch30_ANN_2015_thigh_combined.csv
514
OA71_2016-08-10_08-26-00_to_2016-08-10_13-27-00_Epoch30_ANN_2015_thigh_combined.csv
602
OA71_2016-08-11_10-53-00_to_2016-08-11_18-50-00_Epoch30_ANN_2015_thigh_combined.csv
954
OA71_2016-08-13_07-20-00_to_2016-08-13_22-09-00_Epoch30_ANN_2015_thigh_combined.csv
1778
OA71_2016-08-15_03-41-00_to_2016-08-15_08-00-10_Epoch30_ANN_2015_thigh_combined.csv
514
OA14_2016-09-06_04-04-00_to_2016-09-06_22-28-00_Epoch30_ANN_2015_thigh_combined.csv
2208
OA14_2016-09-05_13-59-46_to_2016-09-05_21-12-00_Epoch30_ANN_2015_thigh_combined.csv
864
OA14_2016-09-07_04-40-00_to_2016-09-07_21-53-00_Epoch30_ANN_2015_thigh_combined.csv
2066
OA14_2016-09-08_04-50-00_to_2016-09-08_21-54-00_Epoch30_ANN_2015_thigh_combined.csv
2048
OA14_2016-09-09_04-15-00_to_2016-09-09_20-30-00_Epoch30_ANN_2015_thigh_combined.csv
1950
OA14_2016-09-10_03-49-00_to_2016-09-10_20-35-00_Epoch30_ANN_2015_thigh_combined.csv
2012
OA14_2016-09-11_04-55-00_t

In [9]:
complete_df = pd.read_csv('/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/thigh_minute_data/per_minute_EE_means_thigh_mont_15.csv')


grouped_df_m1 = complete_df.groupby(['Participant','sex','BMI_range','mont_1_cat']).size().unstack(fill_value=0)
grouped_df_m1.to_csv(join(ActiPAL_output_folder, '3_per_minute_EE_means_thigh_mont_1_grouped.csv'))

grouped_df_m2 = complete_df.groupby(['Participant','sex','BMI_range','mont_2_cat']).size().unstack(fill_value=0)
grouped_df_m2.to_csv(join(ActiPAL_output_folder, '3_per_minute_EE_means_thigh_mont_2_grouped.csv'))

grouped_df_m3 = complete_df.groupby(['Participant','sex','BMI_range','mont_3_cat']).size().unstack(fill_value=0)
grouped_df_m3.to_csv(join(ActiPAL_output_folder, '3_per_minute_EE_means_thigh_mont_3_grouped.csv'))

grouped_df_m4 = complete_df.groupby(['Participant','sex','BMI_range','mont_4_cat']).size().unstack(fill_value=0)
grouped_df_m4.to_csv(join(ActiPAL_output_folder, '3_per_minute_EE_means_thigh_mont_4_grouped.csv'))

In [10]:
def categorize(input_name,output_sheet_name):
  all_df = pd.read_csv(join(ActiPAL_output_folder,input_name))

  female = all_df['sex'] == 'female'
  male = all_df['sex'] == 'male'
  obese = all_df['BMI_range'] == 'obese'
  overweight = all_df['BMI_range'] == 'overweight'
  normal = all_df['BMI_range'] == 'normal'


  male_df = all_df[male]
  female_df = all_df[female]
  obese_df = all_df[obese]
  overweight_df = all_df[overweight]
  normal_df = all_df[normal]

  # print(['all,male,female',len(all_df),len(male_df),len(female_df)])
  # print(['all,male,female',all_df['1'].mean,len(male_df),len(female_df)])

  with pd.ExcelWriter(join(ActiPAL_output_folder,output_sheet_name)) as writer: 
      all_df.to_excel(writer, sheet_name='all')
      female_df.to_excel(writer, sheet_name='female')
      male_df.to_excel(writer, sheet_name='male')
      obese_df.to_excel(writer, sheet_name='obese')
      overweight_df.to_excel(writer, sheet_name='overweight')
      normal_df.to_excel(writer, sheet_name='normal')

In [11]:

categorize('3_per_minute_EE_means_thigh_mont_1_grouped.csv','4_thigh_mont_1_minute_values.xlsx')
categorize('3_per_minute_EE_means_thigh_mont_2_grouped.csv','4_thigh_mont_2_minute_values.xlsx')
categorize('3_per_minute_EE_means_thigh_mont_3_grouped.csv','4_thigh_mont_3_minute_values.xlsx')
categorize('3_per_minute_EE_means_thigh_mont_4_grouped.csv','4_thigh_mont_4_minute_values.xlsx')
